In [3]:
import sys
import os
import argparse
import py3Dmol

def find_list_element_containing_substring(array: list[str, ...], substring):
    for inx, element in enumerate(array):
        if substring in element:
            return inx
    return None

def find_header_structure_split_idx(pdb_file_lines):
    structure_start_idx = -1
    for idx, line in enumerate(pdb_file_lines):
        if line.startswith("ATOM"):
            structure_start_idx = idx
            break
    assert structure_start_idx != -1, "could find ATOM section in given .pdb file. Check .config_ipynb file for path to pdb file"
    return structure_start_idx

def dispatch_sys_argv():
    CONFIG_FILE = '.config_ipynb'
    if os.path.isfile(CONFIG_FILE):
        with open(CONFIG_FILE, "r") as f:
            sys.argv = f.read().split(" ")
            # if there is a space in the .pdb filename, rebuild sys.argv considering optional argument
            if ".pdb" not in sys.argv[1]:
                extension_idx = find_list_element_containing_substring(sys.argv, ".pdb")
                if extension_idx is not None:
                    path = " ".join(sys.argv[1:extension_idx+1])
                    sys.argv = [sys.argv[0], path, *sys.argv[extension_idx+1:]]
    else:
        sys.exit("error")

    arg_parser = argparse.ArgumentParser()
    arg_parser.add_argument("input_file", help="PDB file input")
    arg_parser.add_argument("--mode", help="choose selection mode")
    args = arg_parser.parse_args()
    pdb_file = args.input_file
    selection_mode = args.mode
    return pdb_file, selection_mode


def check_cdr(aa_idx):
    """check if a given aa_idx is in range for CDR and return which cdr"""
    # cdr 1
    if aa_idx in range(26, 37):
        return True, 1

    # cdr 1
    elif aa_idx in range(55, 64):
        return True, 2

    # cdr 1
    elif aa_idx in range(104, 116):
        return True, 3

    # no CDR
    else:
        return False, None



def main():
    pdb_file, selection_mode = dispatch_sys_argv()

    # load pdb
    with open(pdb_file, "r") as ifile:
        raw_pdb_file_content = "".join([x for x in ifile])

    # init view
    view = py3Dmol.view(width=600, height=450)
    view.addModelsAsFrames(raw_pdb_file_content)

    pdb_file_lines = raw_pdb_file_content.split("\n")

    structure_start_idx = find_header_structure_split_idx(pdb_file_lines)
    header, structure = pdb_file_lines[:structure_start_idx], pdb_file_lines[structure_start_idx:]
    chain_number = -1
    first_aa = ""
    for idx, line in enumerate(structure):

        split = line.split()
        if len(split) == 0 or split[0] != "ATOM":
            continue
        # todo add multiple chain color
        # todo not sure if b_factor, occupancy are correct
        # all these idx are 1-based
        is_atom, atom_idx, atom_name, aa_name, a, aa_idx, x, y, z, *_ = split
        atom_idx = int(atom_idx)
        aa_idx = int(aa_idx)
        if aa_idx == 1 and aa_name != first_aa:
            chain_number += 1
            first_aa = aa_name
        coord = [float(x), float(y), float(z)]
        is_cdr, cdr_num = check_cdr(aa_idx)
        if selection_mode.upper() == "CDR":
            color = ["lightblue", "yellow", "red"][cdr_num-1] if is_cdr else ["black", "lightgray"][chain_number]
            view.setStyle({'model': -1, "serial": int(atom_idx)+1}, {"cartoon": {'color': color}})
        else:
            view.setStyle({'model': -1}, {"cartoon": {'color': "spectrum"}})
    # view.setStyle({'model': -1}, {"cartoon": {'color': "spectrum"}})
    view.zoomTo()
    view.show()

if __name__ == "__main__":
    main()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol